<a href="https://colab.research.google.com/github/RV05/NLP-ABSA-research-papers-implementation/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r "/content/drive/MyDrive/absa/data/requirements.txt"

In [ ]:

from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification
from transformers import pipeline

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import base64
import re
from urlextract import URLExtract
extractor = URLExtract()
import contractions
import nltk
import spacy
import json


import tweepy
import csv
import pandas as pd

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
!python -m spacy download en_core_web_lg


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


  Using cached en_core_web_lg-2.3.1-py3-none-any.whl
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
qa_tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
qa_model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
qa_model = pipeline('question-answering', model=qa_model, tokenizer=qa_tokenizer)

sent_tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
sent_model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")
sent_model = pipeline('text-classification', model=sent_model, tokenizer=sent_tokenizer)

In [ ]:
def compute(text, aspects, qa_model, sent_model):
	#preprocessing
	preprocess_text = preprocess(text)
	#get nouns
	noun_list = get_noun(preprocess_text)	
	#get alternative names of aspects
	aspect_classes = get_similar_words(noun_list, aspects)
	#get sentiment
	sentiment_result = get_sentiment(aspect_classes, text, qa_model, sent_model)
	return sentiment_result

In [ ]:
def aspect_sentiment(aspects, hashtag):

	#extracts 50 tweets regarding the hashtag from text
	twitter_content = get_tweets()
	aspect_score = {asp : {'positive': 0, 'negative': 0} for asp in aspects}
	
	#compute scores for each tweet
	if twitter_content.to_dict():
		for text in twitter_content['text']:
			sentiment_result = compute(text, aspects, qa_model, sent_model)
			for result in sentiment_result:
				score = sentiment_result[result]
				if score>0:
					aspect_score[result]['positive'] = aspect_score[result]['positive'] + score
				elif score<0:
					aspect_score[result]['negative'] = aspect_score[result]['negative'] - score
				else:
					pass
	else:
		return {'statusCode': 400, 'body': 'No twitter data scraped for this hashtag'}



	result_list = [[k, 'positive', v['positive']] for k,v in aspect_score.items()]
	result_list.extend([[k, 'negative', v['negative']] for k,v in aspect_score.items()])

	# #plot the bar plot across all aspects
	# aspects_df = pd.DataFrame(result_list, columns= ['aspect', 'sentiment', 'score'])
	# sns.barplot(x = 'aspect', y = 'score', hue='sentiment', data=aspects_df)
	# plt.savefig('result.png')

	# #send base64 string of image as response
	# img_result = None
	# with open('result.png', 'rb') as f:
	# 	im_b64 = base64.b64encode(f.read())
	# 	img_result = str(im_b64)
	# if img_result:
	# 	return {'statusCode': 200, 'body': json.dumps(img_result)}
	# else:
	# 	return {'statusCode': 400, 'body': 'issue in saving result image'}


In [ ]:



def preprocess(text):
    text = str(text).lower()
    #remove urls
    urls = extractor.find_urls(text)
    for url in urls:
        text = text.replace(url, '')
    #remove emails
    text = re.sub(r'\S*@\S*\s?',' ',text)
    #remove mentions
    text = re.sub(r'@\S+', ' ', text)
    #contractions
    text = contractions.fix(text)
    #remove hashtags
    text = re.sub(r'@\S+', ' ', text)
    #remove emojis
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    #remove all punct
    text = re.sub('[^A-z0-9]', ' ', text)
    #remove extras whitespaces
    text = re.sub(' +', ' ', text)
    return text

In [ ]:

def get_noun(lines):
	tokenized = nltk.word_tokenize(lines)
	nouns = set([word for (word, pos) in nltk.pos_tag(tokenized) if(pos[:2] == 'NN')])
	return nouns

# if __name__ == '__main__':
# 	lines = 'battery is not up to the mark but on the other hand screen is good'
# 	result = get_noun(lines)
# 	print(result)

In [ ]:

nlp = spacy.load('en_core_web_lg')

def get_similar_words(nouns, aspects):
	aspect_classes = {k: list() for k in aspects}
	for noun in nouns:
		scores = list()
		for aspect in aspects:
			aspect_token = nlp(aspect)
			noun_token = nlp(noun)
			similarity_score = aspect_token.similarity(noun_token)
			scores.append(similarity_score)
		index = scores.index(max(scores))
		aspect_name = aspects[index]
		
		if max(scores)>0.60:
			value = aspect_classes[aspect_name]
			value.append(noun)
			aspect_classes[aspect_name] = list(set(value))
			
		else:
			pass
	return aspect_classes	





# if __name__ == '__main__':
# 	nouns = ['screen', 'battery']
# 	aspects = ['display', 'battery', 'design', 'iphone13']
# 	result = get_similar_words(nouns, aspects)
# 	print(result)

In [ ]:
Name=[]
context=[]
sentiment1=[]
def get_sentiment(aspect_classes, text, qa_model, sent_model):
  
  sentiment_dict = {k:0 for k in aspect_classes}
  for aspect in aspect_classes:
    alt_names = aspect_classes[aspect]
    for name in alt_names:
      question = f'how is {name}'
      Name.append(name)
      print("question"+question)
      QA_input = {'question': question, 'context': text}
      context.append(text)
      print("QA_input")
      print(QA_input)
      qa_result = qa_model(QA_input)
      print("qa_result")
      print(qa_result)
      answer = qa_result['answer']
      print("answer")
      print(answer)

      #sentiment model 
      sent_result = sent_model(answer)    
      print(sent_result)
      
      sentiment = sent_result[0]['label']
      sentiment1.append(sentiment)
      if sentiment == 'LABEL_0':
        sentiment, score = 'Negative', -1
      elif sentiment == 'LABEL_1':
        sentiment, score = 'Neutral', 0
      else:
        sentiment, score = 'Positive', 1
    
      value = sentiment_dict[aspect] + score
      sentiment_dict[aspect] = value
  return sentiment_dict

In [ ]:
def get_tweets():
    content=pd.read_csv("/content/drive/MyDrive/absa/data/Copy of test - test (1).csv")

    content["text"] = content['text'].str.replace('[^\w\s]',' ')
    content["aspect"] = content['aspect'].str.replace('[^\w\s]',' ')
    content['text'] = content['text'].str.replace('\W', ' ')
    
    return content

In [ ]:
# #Preporcessing
# content=pd.read_csv("/content/drive/MyDrive/AWSLambda_ECR_ABSA-main/Copy of test - test.csv")

# content["text"] = content['text'].str.replace('[^\w\s]',' ')
# content["aspect"] = content['aspect'].str.replace('[^\w\s]',' ')
# content['text'] = content['text'].str.replace('\W', ' ')
# content.head()

,text,aspect
0,improve your customer service and product avai...,Customer service
1,functionality is great almost as in desktop v...,mobile version
2,but it keeps starting from zoomed in and then ...,zoomed
3,hey marilyn thanks for your answer the soc2 ty...,Security
4,delanovc zoom airtable notionhq calendly ...,apple


In [ ]:
# aspects=content['aspect'].tolist()
list1=[]
for aspect1 in content["aspect"]:
  list1.append(aspect1)

aspects=list1
hashtag = 'none'
aspect_sentiment(aspects, hashtag)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  # This is added back by InteractiveShellApp.init_path()


questionhow is customer
QA_input
{'question': 'how is customer', 'context': 'improve your customer service and product availability'}
qa_result
{'score': 0.3745257556438446, 'start': 0, 'end': 29, 'answer': 'improve your customer service'}
answer
improve your customer service
[{'label': 'NEGATIVE', 'score': 0.9852251410484314}]
questionhow is availability
QA_input
{'question': 'how is availability', 'context': 'improve your customer service and product availability'}
qa_result
{'score': 0.6070168614387512, 'start': 0, 'end': 54, 'answer': 'improve your customer service and product availability'}
answer
improve your customer service and product availability
[{'label': 'NEGATIVE', 'score': 0.8150989413261414}]
questionhow is service
QA_input
{'question': 'how is service', 'context': 'improve your customer service and product availability'}
qa_result
{'score': 0.36985209584236145, 'start': 0, 'end': 29, 'answer': 'improve your customer service'}
answer
improve your customer service
[{'lab

KeyboardInterrupt: ignored

In [ ]:
Name

['customer',
 'availability',
 'service',
 'product',
 'version',
 'lot',
 'functionality',
 'speed',
 'music',
 'security',
 'document',
 'copy',
 'answer',
 'type',
 'thanks',
 'zoom',
 'apple',
 'tasks',
 'app',
 'features',
 'customer',
 'care',
 'assistance',
 'project',
 'app',
 'organization',
 'notion',
 'opportunity',
 'background',
 'status']

In [ ]:
context

['improve your customer service and product availability',
 'improve your customer service and product availability',
 'improve your customer service and product availability',
 'improve your customer service and product availability',
 'functionality is great  almost as in desktop version  but mobile version needs a lot of improvement esp the speed',
 'functionality is great  almost as in desktop version  but mobile version needs a lot of improvement esp the speed',
 'functionality is great  almost as in desktop version  but mobile version needs a lot of improvement esp the speed',
 'functionality is great  almost as in desktop version  but mobile version needs a lot of improvement esp the speed',
 'but it keeps starting from zoomed in and then slowly zooming out as the music plays ',
 'hey marilyn thanks for your answer the soc2 type 1 certification is a document that can be shared with external parties for security assessment do you have a copy of the report we could explore ',
 'he

In [ ]:
sentiment1

['NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE']

In [ ]:
import pandas as pd
  
# list of strings

  
# Calling DataFrame constructor after zipping
# both lists, with columns specified
df = pd.DataFrame(list(zip(Name, context,sentiment1)),
               columns =['aspect', 'text','sentiment'])

In [ ]:
df.head()

,aspect,text,sentiment
0,customer,improve your customer service and product avai...,NEGATIVE
1,availability,improve your customer service and product avai...,NEGATIVE
2,service,improve your customer service and product avai...,NEGATIVE
3,product,improve your customer service and product avai...,NEGATIVE
4,version,functionality is great almost as in desktop v...,POSITIVE


In [ ]:
cd /content/drive/MyDrive/absa/data/results

/content/drive/MyDrive/absa/data/results


In [ ]:
df.to_csv('test.csv')